## Hidden Markov Model Class with Neo4j Integration

In [4]:
from py2neo import Graph
import numpy as np
import random

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))


In [5]:
# %% [markdown]
# ## Paper Bag HMM Random Walk Simulation

# %%
class HiddenMarkovModel:
    def __init__(self, graph, hidden_states, observable_states, transition_matrix, emission_matrix, initial_distribution):
        """
        Initializes a Hidden Markov Model (HMM).
        
        :param graph: Neo4j connection
        :param hidden_states: List of hidden states
        :param observable_states: List of observable states
        :param transition_matrix: NxN transition probability matrix
        :param emission_matrix: NxM emission probability matrix
        :param initial_distribution: Initial probability distribution of hidden states
        """
        self.graph = graph
        self.hidden_states = hidden_states
        self.observable_states = observable_states
        self.transition_matrix = np.array(transition_matrix)
        self.emission_matrix = np.array(emission_matrix)
        self.initial_distribution = np.array(initial_distribution)

    def create_hmm(self):
        """Creates the HMM structure in Neo4j."""
        N = len(self.hidden_states)
        M = len(self.observable_states)

        # Clear existing data
        self.graph.run("MATCH (n) DETACH DELETE n")

        # Create hidden states
        for i in range(N):
            self.graph.run(
                "CREATE (:HiddenState {name: $name, initial_prob: $prob})",
                parameters={"name": self.hidden_states[i], "prob": float(self.initial_distribution[i])}
            )

        # Create observable states
        for j in range(M):
            self.graph.run(
                "CREATE (:ObservableState {name: $name})",
                parameters={"name": self.observable_states[j]}
            )

        # Create transitions between hidden states
        for i in range(N):
            for j in range(N):
                prob = float(self.transition_matrix[i][j])
                if prob > 0:
                    self.graph.run(
                        """
                        MATCH (a:HiddenState {name: $from}), (b:HiddenState {name: $to})
                        CREATE (a)-[:TRANSITION {probability: $prob}]->(b)
                        """,
                        parameters={"from": self.hidden_states[i], "to": self.hidden_states[j], "prob": prob}
                    )

        # Create emissions from hidden states to observable states
        for i in range(N):
            for j in range(M):
                prob = float(self.emission_matrix[i][j])
                if prob > 0:
                    self.graph.run(
                        """
                        MATCH (h:HiddenState {name: $from}), (o:ObservableState {name: $to})
                        CREATE (h)-[:EMITS {probability: $prob}]->(o)
                        """,
                        parameters={"from": self.hidden_states[i], "to": self.observable_states[j], "prob": prob}
                    )

        print("HMM created in Neo4j.")

    def simulate_random_walk(self, start_state=None, steps=10):
        """
        Simulates a random walk in the Markov Chain.
        
        :param start_state: Starting hidden state (defaults to one chosen by initial distribution)
        :param steps: Number of steps in the walk
        """
        if sum(self.initial_distribution) == 0:
            raise ValueError("Initial distribution must sum to greater than zero.")

        if start_state is None:
            # Choose starting state based on initial probabilities
            current_state = random.choices(self.hidden_states, weights=self.initial_distribution)[0]
        else:
            current_state = start_state

        for step in range(steps):
            print(f"Step {step + 1}: Current Hidden State -> {current_state}")

            # Emit an observable state
            state_index = self.hidden_states.index(current_state)
            emission = random.choices(self.observable_states, weights=self.emission_matrix[state_index])[0]
            print(f"  Emitted Observation: {emission}")

            # Transition to the next hidden state
            next_state = random.choices(self.hidden_states, weights=self.transition_matrix[state_index])[0]
            current_state = next_state

        print(f"Final Hidden State: {current_state}")


In [6]:
# Define Paper Bag HMM parameters
hidden_states_paperbag = ['A', 'B']  # The bags (Hidden states)
observable_states_paperbag = ['j', 'k']  # The chips (Observable states)

# Transition probabilities (Bag -> Bag)
transition_matrix_paperbag = [
    [0.40, 0.60],  # If in Bag A: 5% stay in A, 95% switch to B
    [0.80, 0.20]   # If in Bag B: 80% stay in B, 20% switch to A
]

# Emission probabilities (Bag -> Chip drawn)
emission_matrix_paperbag = [
    [4/5, 1/5],  # If in Bag A: 80% chance to draw 'j', 20% chance to draw 'k'
    [1/5, 4/5]   # If in Bag B: 20% chance to draw 'j', 80% chance to draw 'k'
]

# Initial probability distribution (starting in Bag A)
initial_distribution_paperbag = [1.0, 0.0]  # Always start in Bag A

# Create and load the Paper Bag HMM into Neo4j
hmm_paperbag = HiddenMarkovModel(
    graph,
    hidden_states_paperbag,
    observable_states_paperbag,
    transition_matrix_paperbag,
    emission_matrix_paperbag,
    initial_distribution_paperbag
)
hmm_paperbag.create_hmm()

# Run the random walk simulation
hmm_paperbag.simulate_random_walk(steps=10)


HMM created in Neo4j.
Step 1: Current Hidden State -> A
  Emitted Observation: j
Step 2: Current Hidden State -> B
  Emitted Observation: k
Step 3: Current Hidden State -> A
  Emitted Observation: j
Step 4: Current Hidden State -> B
  Emitted Observation: k
Step 5: Current Hidden State -> A
  Emitted Observation: j
Step 6: Current Hidden State -> B
  Emitted Observation: j
Step 7: Current Hidden State -> A
  Emitted Observation: j
Step 8: Current Hidden State -> B
  Emitted Observation: k
Step 9: Current Hidden State -> A
  Emitted Observation: j
Step 10: Current Hidden State -> A
  Emitted Observation: j
Final Hidden State: B
